In [91]:
from glob import glob
from PIL import Image,ImageFilter
from skimage.morphology import skeletonize,square
import numpy as np
import pandas as pd 

from PVBM.GeometricalAnalysis import GeometricalVBMs
from PVBM.FractalAnalysis import MultifractalVBMs

In [92]:
paths_list = glob('data/data_m_vessel/*/*/*png') + glob('data/data_m_vessel/*/*/*tif')

In [100]:
pvbm_data_ba_t = pd.DataFrame({'image_name': [],
                          'group': [],
                          'status': [],
                          'median_tortuosity': [],
                          'tortuosities': [],
                          'tortuosity_index': [],
                          'angles': [],
                          'BA_mean': [],
                          'BA_std': [],
                          'BA_median':[]})

pvbm_data_ba_t['tortuosities'] = pvbm_data_ba_t['tortuosities'].astype(object)
pvbm_data_ba_t['angles'] = pvbm_data_ba_t['angles'].astype(object)

In [101]:
pvbm_data_ba_t = pd.read_csv('morphological_data/morph_data_ba_t.csv', index_col=0)
pvbm_data_ba_t 

,image_name,group,status,median_tortuosity,tortuosities,tortuosity_index,angles,BA_mean,BA_std,BA_median
1,178_N.png,test,0.0,1.081524,"[1.104469180660218, 1.0853682063011378, 1.0808...",1.110732,"[81.02737338510362, 80.06361005657166, 146.232...",103.657447,42.680454,108.913930
2,193_N.png,test,0.0,1.077366,"[1.070019809918359, 1.0653784275844873, 1.0424...",1.085169,"[83.20848267583968, 81.86989764584403, 84.9575...",95.715414,40.722771,92.205188
3,453_N.png,test,0.0,1.081569,"[1.0647331072635022, 1.0843587464779227, 1.107...",1.100095,"[113.96248897457819, 112.83365417791754, 77.00...",100.286770,42.567115,103.100874
4,479_N.png,test,0.0,1.077593,"[1.0810570087511775, 1.0866427919377388, 1.051...",1.079668,"[79.9739629096787, 151.01543960626566, 151.247...",97.073281,44.409226,94.527667
5,486_N.png,test,0.0,1.079669,"[1.0815530272613485, 1.0547086412307767, 1.080...",1.084416,"[81.83335697508824, 74.55168816755315, 145.203...",97.800842,47.523901,95.452101
...,...,...,...,...,...,...,...,...,...,...
539,01_dr.tif,train,1.0,1.079669,"[1.1410619601039307, 1.1263259861698796, 1.089...",1.096740,"[66.99740340664215, 64.66472300780069, 176.443...",98.439450,47.383950,95.425226
540,02_dr.tif,train,1.0,1.079214,"[1.1221204005577847, 1.0962771553989088, 1.076...",1.090571,"[109.63678188737414, 56.61673425127014, 142.85...",99.532030,44.342424,91.118431
541,03_dr.tif,train,1.0,1.079361,"[1.0817692691420735, 1.0750713750238035, 1.025...",1.093113,"[129.13039955698528, 140.0796078600146, 10.949...",101.503812,45.310385,96.890610
542,04_dr.tif,train,1.0,1.079228,"[1.0810175771816928, 1.0822096652118491, 1.110...",1.091747,"[82.17939243177618, 78.18685529986284, 160.662...",96.591480,44.044070,89.215175


In [102]:
for index, path in enumerate(paths_list, start=1):
    if index in range(1,pvbm_data_ba_t.index[-1]): continue
    
    pp = path.split('\\') # example ['data/data_m_vessel', 'test', 'control', '10_h.tif']
    pvbm_data_ba_t.at[index, 'image_name'] = pp[-1]
    if pp[-2] == 'control':
        pvbm_data_ba_t.at[index, 'status'] = 0 #healthy
    else:
        pvbm_data_ba_t.at[index, 'status'] = 1 #diabetic
    pvbm_data_ba_t.at[index, 'group'] = pp[1]
    
        
    segmentation = Image.open(path)
    segmentation = segmentation.filter(ImageFilter.ModeFilter(size=3))
    segmentation = np.array(segmentation)/255
    if len(segmentation.shape) == 3 : segmentation = segmentation[:,:,0]
    skeleton = skeletonize(segmentation) 
    
    geometricalVBMs = GeometricalVBMs()
    
    median_tor, _,chord,arc,_ = geometricalVBMs.compute_tortuosity_length(skeleton)
    pvbm_data_ba_t.at[index, 'median_tortuosity'] = median_tor
    pvbm_data_ba_t.at[index, 'tortuosity_index'] = np.sum(arc)/np.sum(chord)
    pvbm_data_ba_t.at[index, 'tortuosities'] = (np.array(arc)/np.array(chord)).tolist()

    mean_ba,std_ba,median_ba,angle_dico,_ = geometricalVBMs.compute_branching_angles(skeleton)
    pvbm_data_ba_t.at[index, 'BA_mean'] = mean_ba
    pvbm_data_ba_t.at[index, 'BA_std'] = std_ba
    pvbm_data_ba_t.at[index, 'BA_median'] = median_ba
    pvbm_data_ba_t.at[index, 'angles'] = list(angle_dico.values())
    
pvbm_data_ba_t.to_csv('morphological_data/morph_data_ba_t.csv', index=True)    

In [103]:
pvbm_data_ba_t.to_csv('morphological_data/morph_data_ba_t.csv', index=True)    